# 템플릿 시간표 변경

## 라이브러리 불러오기

In [1]:
import getpass
import os
import pandas as pd
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


## 시간표 변경 템플릿

In [8]:
prompt_template = """
너는 지금부터 블로그 포스팅 전문 콘텐츠 마케터야.
너는 항상 최선을 다하고 좋은 글을 작성해서 나를 기쁘게 해주고 있어.
아래의 형식으로 작성해줘:

{input}

1) 제목과 본문으로 구분해서 출력해줘. 
2) 제목은 창의력 있고, 주목도 있게 구성해줘. 
3) 신뢰도 있는 정보를 중심으로 포스팅을 해줘 
4) 템플릿에 해당하 정보가 input에 없으면 생략해줘


템플릿:
{{여기에 블로그 제목}}
{{시작하는 말}}
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

{{변경 노선}}
{{변경 일시}}
{{첫차/막차 시간}}
{{문의 사항 링크}}


{{마무리 말}}
오지통이 실시간으로 다양한 지하철 정보를업데이트 할 예정이니, 자주 방문해 주세요. 
'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개 보러가기
https://blog.naver.com/subway__onda/223258646349
"""

## 평가 템플릿

In [4]:
Consistency_template = """

Compare and evaluate the blog content you created with the article.
First, Extract the key information from the article ( time,date,subway line)
You need to decide whether the key information is entailed by the  CONTEXT by choosing one of the following rating: 

1. 5 point: The blog content follows logically from the information contained in the article. 

2. 1 point: The blog content is logically false from the information contained in the article. 

3. an integer score between 1 and 5 and if such integer score does not exist,  

use 1: It is not possible to determine whether the blog content is true or false without further information. 

Read the passage of information thoroughly and select the correct answer from the three answer labels. 

Read the CONTEXT thoroughly to ensure you know what the CONTEXT entails.  

Note the blog content is generated by a computer system, it can contain certain symbols, which should not be a negative factor in the evaluation.


blog content
{input1}

article
{input2}

Evaluation Form (scores ONLY):

{{score}}

"""

In [5]:
Coherence_template = """

Your task is to rate the blog content on one metric.

Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

Coherence (1-5) - the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby "the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic."

Evaluation Steps:

1. Read the news article carefully and identify the main topic and key points.
2. Read the blog content and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.


{input}

Evaluation Form (scores ONLY):


{{score}}


"""

In [6]:
Friendliness_template = """
Friendliness, as a metric, measures how effectively the text of a blog content engages the reader with a friendly and inviting tone. If the tone and style of the text foster a sense of warmth and familiarity, making the reader feel welcomed and engaged, then the value of the Friendliness metric should be high; otherwise, it should be low. After reading the blog post, evaluate its tone and style against typical friendly communication, and determine the value of the Friendliness metric using the following rating scale:

- One star: The tone is cold or unfriendly, not engaging the reader in any appreciable way.
- Two stars: The tone has minimal warmth, slightly engaging but largely formal or distant.
- Three stars: The tone is moderately friendly, balancing formal and informal elements to somewhat engage the reader.
- Four stars: The tone is very friendly, actively engaging the reader with warmth and lively expressions.
- Five stars: The tone is exceptionally friendly and lively, creating a strong sense of connection and engagement with the reader.

This rating should always be an integer between 1 and 5. Thus, the rating produced should be 1, 2, 3, 4, or 5, based on your assessment of how friendly and engaging the text is towards the read.


{input}

Evaluation Template (Score Only):


{{score}}


"""

In [7]:
Fluency_template = """
Fluency measures the quality of individual sentences in the answer, and whether they are well-written and grammatically correct. Consider the quality of individual sentences when evaluating fluency. Given the question and answer, score the fluency of the answer between one to five stars using the following rating scale: 

One star: the answer completely lacks fluency 

Two stars: the answer mostly lacks fluency 

Three stars: the answer is partially fluent 

Four stars: the answer is mostly fluent 

Five stars: the answer has perfect fluency 

This rating value should always be an integer between 1 and 5. So the rating produced should be 1 or 2 or 3 or 4 or 5.


{input}


Evaluation Template (Score Only):


{{score}}



"""

In [8]:
Humanlikeness_template = """
Humanlikeness, as a metric, evaluates how closely the style and delivery of GPT-generated text resemble that of naturally written human text. If the text is indistinguishable from human writing in terms of natural flow, vocabulary use, and contextual accuracy, then the value of the Human-likeness metric should be high; otherwise, it should be low. Given the text generated by GPT and comparing it to typical human-written text, determine the value of the Human-likeness metric using the following rating scale:

- One star: The text is entirely machine-like, with unnatural flow and numerous contextual errors.
- Two stars: The text is mostly machine-like, with frequent unnatural expressions and noticeable errors.
- Three stars: The text is somewhat human-like, showing signs of natural language use but with some mechanical characteristics.
- Four stars: The text is mostly human-like, with very few mechanical traits and generally natural language use.
- Five stars: The text is completely human-like, indistinguishable from something a human might write.

This rating should always be an integer between 1 and 5. Thus, the rating produced should be 1, 2, 3, 4, or 5, based on the evaluation.


{input}

Evaluation Template (Score Only):

{{score}}

"""

In [9]:
Readability_template = """ 
To assess the readability of an article, please read and evaluate the presented content carefully. The readability assessment focuses on three main factors

1. Sentence length: The sentence should not be excessively long.
2. Vocabulary difficulty: We prefer to use everyday, easy-to-understand vocabulary.
3. Line breaks and paragraph breaks: A clear structure improves readability.

Based on each factor, please rate the overall readability on a integer from 1 to 5. The scoring system is as follows:

*  1 point: Very long sentences, difficult vocabulary, and no line breaks or paragraph breaks.

*  5 point: Sentence length is reasonable, uses everyday words that are easy to understand, and has good line breaks and paragraph breaks.


{input}

Evaluation Template (Score Only):

{{score}}

"""

## 실험

In [9]:
# prompt + model + output parser
llm = ChatOpenAI(model="gpt-4-turbo")
timeprompt = ChatPromptTemplate.from_template(prompt_template)
output_parser = StrOutputParser()

# LCEL chaining
timechain = timeprompt | llm | output_parser

## 텍스트로 주입

In [16]:
text1 = """
제목	1호선 열차운영계획 추가 조정 알림(경인급행열차)
작성일	2024-04-24	
내용	
1호선 열차운영계획 추가 조정 알림(경인급행열차)



1호선 경인급행열차(용산↔동인천 구간) 열차운행계획 추가 조정을

아래와 같이 안내드리오니 열차 이용에 참고하시기 바랍니다.



가. 시행일시: '24. 5. 1.(수)

나. 조정호선: 1호선(경인급행)

다. 열차운행 시각표: 붙임 참고
"""

In [17]:
text2 = """
제목	신분당선 열차운행시각표 변경 시행
등록일	2023-06-21	
첨부파일	파일첨부 20230701 평일 역별 열차시각표.pdf
■ 변경대상 : 신사역 ~ 광교역



■ 변경사유 : 신분당선 열차운행계획 변경



■ 시행일자 : 2023.7.1(토) 영업개시부터 적용



※ 토요일 및 휴일 열차운행시각표 변경 없음



■ 역사별 열차시각표는 첨부파일을 다운로드 받으시기 바랍니다.
"""

In [18]:
text3 = """
공항철도 열차시각 변경 알림(5/1~)

등록일 2023-04-20 16:01:13.0
출근시간대 혼잡도 완화를 위해 열차시각이 변경됨을 알려드리오니 열차이용에 참고하시기 바랍니다.
 
가. 변   경   일 : 2023. 5. 1.(월)
나. 변경 내용 : 평일 일반열차, 직통열차 열차운행시각 전면 개정(주말, 공휴일은 기존과 동일)
다. 시   각   표 : 첨부파일 참고


"""

In [19]:
text4 = """
2023년 제야의 종 타종행사 개최 관련 연장운행 안내
2023.12.29 
2023년 제야의 종 타종행사 관련
지하철 연장운행 안내

연말 제야의 종 타종행사에 참석하시는 시민의 교통편의를 위하여
아래와 같이 운행시간을 연장하오니
우이신설도시철도를 이용하시기 바랍니다.

○ 행사일시 : 2023. 12. 31(일) 23:00 ~ 2024. 01. 01(월) 01:00
○ 행사장소 : 보신각 일대
○ 지하철 연장운행 안내
 - 운행일시 : 2023. 12. 31(일) 영업운행 기준 다음날 02:05까지 운행
               (종착역 도착시각 기준)
 - 운행구간 : 우이신설선 북한산우이역 ~ 신설동역
 - 운행시격 : 12 ~ 30분 간격
 - 역사별 운행시각표 (아래표 참고)
 
"""

In [20]:
text5 = """

7호선 열차운행 시각표 변경 안내


작성일	2024년 4월 30일(화) 17:44:45
 

7호선 열차운행 시각표 변경 안내

 

 

7호선 열차운행 시각표 변경 사항을 아래와 같이 알려드리오니 열차 이용에 참고하시기 바랍니다.

 

 

         ♦ 시행일자 : 2024. 5. 1.(수) 부터

         ♦ 대상호선 : 7호선

         ♦ 내       용 : 서울구간 퇴근시간대 2회 증회 관련 열차운행 시각표 일부 변경

          ※ 변경된 열차시각표는 첨부파일을 참조하시기 바랍니다.
"""

## 실험결과

In [21]:
output1 = timechain.invoke({"input": text1})
output2 = timechain.invoke({"input": text2})
output3 = timechain.invoke({"input": text3})
output4 = timechain.invoke({"input": text4})
output5 = timechain.invoke({"input": text5})

In [22]:
# 입력값 리스트
inputs = [
    (output1, text1),
    (output2, text2),
    (output3, text3),
    (output4, text4),
    (output5, text5)
]

In [24]:
# 평가 항목 / 템플릿
evaluations = {
    'Consistency': Consistency_template,
    'Coherence': Coherence_template,
    'Friendliness': Friendliness_template,
    'Fluency': Fluency_template,
    'Humanlikeness': Humanlikeness_template,
    'Readability': Readability_template
}

# 결과 저장
results = []

for input1, input2 in inputs:
    scores = {}
    for eval_name, template in evaluations.items():
        if eval_name == 'Consistency':
            # Consistency의 경우에는 원본글과 블로그 글 두 입력값이 필요
            response = llm.invoke(template.format(input1=input1, input2=input2))
        else:
            response = llm.invoke(template.format(input=input1))
            
        scores['text'] = input2
        scores['output'] = input1
        score = response.content
        scores[eval_name] = score
    
    # 현재 입력에 대한 점수를 결과 리스트에 추가
    results.append(scores)

# 결과 리스트로 DataFrame 생성
df_time = pd.DataFrame(results)
df_time


,text,output,Consistency,Coherence,Friendliness,Fluency,Humanlikeness,Readability
0,\n제목\t1호선 열차운영계획 추가 조정 알림(경인급행열차)\n작성일\t2024-0...,"**경인급행, 새로운 시간표로 더 빠르게! 1호선 열차운행계획 추가 조정 소식**\...",5,Coherence: 4,4,5,4,**Evaluation:**\n\n1. **Sentence length:** The...
1,\n제목\t신분당선 열차운행시각표 변경 시행\n등록일\t2023-06-21\t\n첨...,"**[새로운 아침! 신분당선 열차운행 변경소식!]**\n\n안녕하세요, 여러분의 출...",{score: 5},Coherence: 4,4,5,4,Based on the three main factors for readabilit...
2,\n공항철도 열차시각 변경 알림(5/1~)\n\n등록일 2023-04-20 16:0...,"**새벽 출근길 스트레스, 이제 그만! 공항철도 시간표 대변신🚄**\n\n안녕하세요...",5,4,4,5,4,4
3,\n2023년 제야의 종 타종행사 개최 관련 연장운행 안내\n2023.12.29 \...,"**제야의 종과 함께하는 밤, 우이신설선 연장 운행으로 걱정 끝!**\n\n안녕하세...",5,Coherence: 4\n\nThe blog content effectively c...,4,5,4,4
4,\n\n7호선 열차운행 시각표 변경 안내\n\n\n작성일\t2024년 4월 30일(...,"**7호선, 퇴근길이 달라진다! 증회 열차로 더 빠르게 집에 가요!**\n\n안녕하...",5,Coherence: 4,4,5,4,**Evaluation:**\n\n1. **Sentence length:** The...


In [25]:
df_time.to_csv("./timeresult.csv")

# 실제 글 발행 0528

In [5]:
input = """
제목	8호선 영업시운전 관련 열차운행계획 조정 안내 (5/25~6/16)
작성일	2024-05-21	
내용	
8호선 영업시운전 관련 열차시각표 조정 안내



8호선 별내선 구간의 영업시운전 계획에 따라, 열차운행계획 조정을

아래와 같이 안내드리오니 열차 이용에 참고하시기 바랍니다.



가. 시행기간 : '24. 5. 25.(토) ~ 6. 16.(일) / 23일간

나. 조정호선 : 8호선(평일, 휴일)

다. 열차운행 시각표: 붙임 참고



※ [별내선 구간(암사역사공원역~별내역)] 구간은 영업시운전으로 인해 이용 승객의 승하차는 불가합니다.
"""

In [6]:
output_0528 = timechain.invoke({"input": input})

In [7]:
print(output_0528)

**🚆 8호선 영업시운전! 별내선 구간 이용 불가 안내 🚧**

안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

지하철 이용이 일상인 우리에게, 조금은 특별한 소식이 있어 왔어요. 바로 8호선 별내선 구간의 영업시운전 소식인데요, 이로 인해 열차 운행계획에 조정이 생겼습니다. 지하철 파업이 아니어서 다행이지만, 이번 조정도 출퇴근에 영향을 줄 수 있으니 주목해주세요!

**변경 노선:** 8호선 (평일, 휴일 포함)

**변경 일시:** '24. 5. 25.(토) ~ 6. 16.(일) / 23일간

**첫차/막차 시간:** 상세 열차운행 시각표는 붙임파일을 참고해주세요.

🔽 **별내선 구간(암사역사공원역~별내역)**은 영업시운전으로 인해 승하차가 불가합니다. 이 구간을 이용하시는 분들은 반드시 다른 교통수단을 이용하셔야 해요. 불편을 겪을 여러분께는 미리 양해의 말씀 드립니다!

오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개 보러가기
https://blog.naver.com/subway__onda/223258646349


In [10]:
output_0528_2 = timechain.invoke({"input": input})
print(output_0528_2)

**[주목!] 8호선 별내선 영업시운전으로 인한 열차운행계획 조정 안내 (5/25~6/16)**

안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

**변경 노선**: 8호선 별내선  
**변경 일시**: '24년 5월 25일(토)부터 6월 16일(일)까지  
**첫차/막차 시간**: 열차운행 시각표는 붙임 파일을 참고해 주세요.  

8호선 별내선 구간은 이번 영업시운전 기간 동안, 암사역사공원역부터 별내역 구간에서 승하차가 불가능합니다. 이 점 유의하여 승객 여러분들께서는 불편 없이 다른 교통수단을 이용하시길 바랍니다.

오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요.  
'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개 보러가기  
[지하철 온다 블로그 링크](https://blog.naver.com/subway__onda/223258646349)
